# Evaluation of RAG Using Ragas

In the following notebook we'll explore how to evaluate RAG pipelines using a powerful open-source tool called "Ragas". This will give us tools to evaluate component-wise metrics, as well as end-to-end metrics about the performance of our RAG pipelines.

In the following notebook we'll complete the following tasks:

The only way to get started is to get started - so let's grab our dependencies for the day!

> NOTE: Using this notebook as presented will incur a charge of ~$3USD from OpenAI usage.

- 🤝 Breakout Room #1
  1. Task 1: Installing Required Libraries
  2. Task 2: Set Environment Variables
  3. Task 3: Creating a Simple RAG Pipeline with LangChain v.0.2.0
  4. Task 4: Synthetic Dataset Generation for Evaluation using Ragas (Optional)

- 🤝 Breakout Room #2
  1. Task 1: Evaluating our Pipeline with Ragas
  2. Task 2: Testing OpenAI's Claim
  3. Task 3: Selecting an Advanced Retriever and Evaluating

> NOTE: This Notebook *does* contain a bonus challenge, outlined at the bottom of the notebook, which you can complete instead of the notebook for full marks on the assignment.

## Motivation

A claim, made by OpenAI, is that their `text-embedding-3-small` is better (generally) than their `text-embedding-ada-002` model.

Here's some passages from their [blog](https://openai.com/blog/new-embedding-models-and-api-updates) about the `text-embedding-3` release:

> `text-embedding-3-small` is our new highly efficient embedding model and provides a significant upgrade over its predecessor, the `text-embedding-ada-002` model...

> **Stronger performance.** Comparing `text-embedding-ada-002` to `text-embedding-3-small`, the average score on a commonly used benchmark for multi-language retrieval ([MIRACL](https://github.com/project-miracl/miracl)) has increased from 31.4% to 44.0%, while the average score on a commonly used benchmark for English tasks ([MTEB](https://github.com/embeddings-benchmark/mteb)) has increased from 61.0% to 62.3%.

Well, with a library like Ragas - we can put that claim to the test!

If what they claim is true - we should see an increase on related metrics by using the new embedding model!

# 🤝 Breakout Room Part #1

## Task 1: Installing Required Libraries

A reminder that one of the [key features](https://python.langchain.com/v0.2/docs/versions/v0_2/) of LangChain v0.2.0 is the compartmentalization of the various LangChain ecosystem packages!

So let's begin grabbing all of our LangChain related packages!

In [2]:
!pip install -U -q langchain langchain-openai langchain_core langchain-community langchainhub openai langchain-qdrant

We'll also get the "star of the show" today, which is Ragas!

In [3]:
!pip install -qU ragas

We'll be leveraging [QDrant](https://qdrant.tech/) again as our LangChain `VectorStore`.

We'll also install `pymupdf` and its dependencies which will allow us to load PDFs using the `PyMuPDFLoader` in the `langchain-community` package!

In [4]:
!pip install -qU qdrant-client pymupdf pandas

## Task 2: Set Environment Variables

Let's set up our OpenAI API key so we can leverage their API later on.

In [5]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

In [50]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

## Task 3: Creating a Simple RAG Pipeline with LangChain v0.2.0

Building on what we've been learning, we'll be leveraging LangChain v0.2.0 and LCEL to build a simple RAG pipeline that we can baseline with Ragas.

## Building our RAG pipeline

Let's review the basic steps of RAG again:

- Create an Index
- Use retrieval to obtain pieces of context from our Index that are similar to our query
- Use a LLM to generate responses based on the retrieved context

Let's get started by creating our index.

> NOTE: We're going to start leaning on the term "index" to refer to our `VectorStore`, `VectorDatabase`, etc. We can think of "index" as the catch-all term, whereas `VectorStore` and the like relate to the specific technologies used to create, store, and interact with the index.

### Creating an Index

You'll notice that the largest changes (outside of some import changes) are that our old favourite chains are back to being bundled in an easily usable abstraction.

We can still create custom chains using LCEL - but we can also be more confident that our pre-packaged chains are creating using LCEL under the hood.

#### Loading Data

Let's start by loading some data!

- [`PyMuPDFLoader`](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyMuPDFLoader.html)

> NOTE: You'll notice that we're using a document loader from the community package of LangChain. This is part of the v0.2.0 changes that make the base (`langchain-core`) package remain lightweight while still providing access to some of the more powerful community integrations.

In [6]:
from langchain_community.document_loaders import PyMuPDFLoader

PDF_LINK = "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf"

loader = PyMuPDFLoader(
   file_path=PDF_LINK
)

documents = loader.load()

In [7]:
documents[0].metadata

{'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf',
 'page': 0,
 'total_pages': 195,
 'format': 'PDF 1.3',
 'title': 'The Pmarca Blog Archives',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'Mac OS X 10.10 Quartz PDFContext',
 'creationDate': "D:20150110020418Z00'00'",
 'modDate': "D:20150110020418Z00'00'",
 'trapped': ''}

#### Transforming Data

Now that we've got our single document - let's split it into smaller pieces so we can more effectively leverage it with our retrieval chain!

We'll start with the classic: `RecursiveCharacterTextSplitter`.

- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html#langchain-text-splitters-character-recursivecharactertextsplitter)

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

CHUNK_SIZE = 200
CHUNK_OVERLAP = 50

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = CHUNK_SIZE,
    chunk_overlap = CHUNK_OVERLAP
)

documents = text_splitter.split_documents(documents)

Let's confirm we've split our document.

In [9]:
len(documents)

1864

#### Loading OpenAI Embeddings Model

We'll need a process by which we can convert our text into vectors that allow us to compare to our query vector.

Let's use OpenAI's `text-embedding-ada-002` for this task!

- [`OpenAIEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html#langchain-openai-embeddings-base-openaiembeddings)

> NOTE: We are purposefully using an older embedding model to try and answer the guiding question: Is TE3 better than Ada-002?

In [10]:
from langchain_openai import OpenAIEmbeddings

EMBEDDING_MODEL = "text-embedding-ada-002"

embeddings = OpenAIEmbeddings(
    model=EMBEDDING_MODEL
)

#### Creating a QDrant VectorStore

Now that we have documents - we'll need a place to store them alongside their embeddings.

- [`Qdrant`](https://api.python.langchain.com/en/latest/qdrant/langchain_qdrant.qdrant.QdrantVectorStore.html#langchain_qdrant.qdrant.QdrantVectorStore)

> NOTE: You'll need to provide the embedding dimension for Ada-002!

In [11]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

LOCATION = ":memory:"
COLLECTION_NAME = "PMarca Blogs"
VECTOR_SIZE = 1536

In [12]:
qdrant_client = QdrantClient(
    location=LOCATION
)

qdrant_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=VECTOR_SIZE, distance=Distance.COSINE)
)

qdrant_vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME,
    embedding=embeddings
)

qdrant_vector_store.add_documents(documents)

['c49bf216a2594bbf891e841c96250e57',
 'a1602171df614162bcdc3493ccd51d58',
 '86c6a2b2c53c4f36890d3fc76f20a4bb',
 'da1b61da73034a40be56c02e3a25b56c',
 '944b8fbc8f6f4e9994fa8b8cdd03f887',
 '84e02cd33d604384aed2d6e2d40cb867',
 'd8881beef1184afc9123198e7276aee9',
 '8c071040808a49bbbfdb405833bf2949',
 '2c1379e1b60a42b6a866117d2170fed5',
 '305dd22e830d462588db01d48f09b6f2',
 '2e8a7a8f33c64b12b797006ca162d63c',
 '5580878b6feb4500a3b8556a4c9510c8',
 'a50ca46a4d484679bca217c2b473709b',
 '5dd75a6e4d434c59a4278ba93cc1a3ef',
 'c2350e49312142bba8a5e580559da5bc',
 '60dc5b19fce547579d1ecb4da5eeb4eb',
 '58ecac18254e4a2fb45f3ef9e532951a',
 'bc275a049c5e4beb9d1432fa2cad9f3d',
 'e526006ed1684b8890e1de12b6f393cf',
 'd1dd69832dae45cbaa7fb17b0a514baf',
 'c9797fa08c564a609dd137e561b6b9d2',
 '04006af084a5452a9bacebce4067c44d',
 'f520e565f7404e9486a5ddd88a55d611',
 '61e6a245831540a29eca4b8d8f73c55f',
 '9d185eb27a73438f86d01e98ba790821',
 'd969c142aa3c448d99f30fdf16fe7b27',
 '710a7b7a20534e30a30ad1cb7a581a8b',
 

####❓ Question #1:

List out a few of the techniques that Qdrant uses that make it performant.

> NOTE: Check the [documentation](https://qdrant.tech/documentation/overview/) for more information about QDrant!

--> Qdrant uses several techniques for performance:
1. HNSW (Hierarchical Navigable Small World) for fast approximate nearest neighbor search
2. Vector quantization for efficient storage and retrieval
3. In-memory storage with disk persistence for speed
4. Parallel processing for query execution
5. Optimized index structures for fast filtering

#### Creating a Retriever

To complete our index, all that's left to do is expose our vectorstore as a retriever - which we can do the same way we would in previous version of LangChain!

In [13]:
retriever = qdrant_vector_store.as_retriever()

#### Testing our Retriever

Now that we've gone through the trouble of creating our retriever - let's see it in action!

In [14]:
retrieved_documents = retriever.invoke("What is a rule of thumb for selecting an industry to invest in?")

In [15]:
for doc in retrieved_documents:
  print(doc)

page_content='the existing order — and make sure that those forces of change
have a reasonable chance at succeeding.
Second rule of thumb:
Once you have picked an industry, get right to the center of it' metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 125, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '2f3dd87a69e64d07a7864ebd05e0592e', '_collection_name': 'PMarca Blogs'}
page_content='Third rule:
In a rapidly changing Held like technology, the best place to
get experience when you’re starting out is in younger, high-
growth companies.' metadata={'source': 'https://d1lamhf6l6yk6d.cloud

### Creating a RAG Chain

Now that we have the "R" in RAG taken care of - let's look at creating the "AG"!

#### Creating a Prompt Template

There are a few different ways we could create our prompt template - we could create a custom template, as seen in the code below, or we could simply pull a prompt from the prompt hub! Let's look at an example of that!

In [51]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [52]:
print(retrieval_qa_prompt.messages[0].prompt.template)

Answer any use questions based solely on the context below:

<context>
{context}
</context>


As you can see - the prompt template is simple (and has a small error) - so we'll create our own to be a bit more specific!

In [19]:
from langchain.prompts import ChatPromptTemplate

template = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Question:
{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_template(template)

#### Setting Up our Basic QA Chain

Now we can instantiate our basic RAG chain!

We'll use LCEL directly just to see an example of it - but you could just as easily use an abstraction here to achieve the same goal!

We'll also ensure to pass-through our context - which is critical for RAGAS.

In [20]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

retrieval_augmented_qa_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

Let's test it out!

In [21]:
question = "What is a rule of thumb for selecting an industry to invest in?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)

A rule of thumb for selecting an industry to invest in is to ensure that the forces of change within that industry have a reasonable chance at succeeding. Additionally, once you have picked an industry, it is advised to get right to the center of it, where the great opportunities can be found.


In [22]:
question = "What did Pink Floyd have to say about how to proceed when investing in a new industry?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

I don't know.
[Document(metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'page': 15, 'total_pages': 195, 'format': 'PDF 1.3', 'title': 'The Pmarca Blog Archives', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Mac OS X 10.10 Quartz PDFContext', 'creationDate': "D:20150110020418Z00'00'", 'modDate': "D:20150110020418Z00'00'", 'trapped': '', '_id': '90fc874d22dc4b27ab8f24e5bd72182e', '_collection_name': 'PMarca Blogs'}, page_content='ask if you can call them again if things change.\nTrust me — they’d much rather be saying “yes” than “no” —\nthey need all the good investments they can get.\nSecond, consider the environment.'), Document(metadata={'source': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf', 'file_path': 'https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-

We can already see that there are some improvements we could make here.

For now, let's switch gears to RAGAS to see how we can leverage that tool to provide us insight into how our pipeline is performing!

## Task 4: Synthetic Dataset Generation for Evaluation using Ragas

Ragas is a powerful library that lets us evaluate our RAG pipeline by collecting input/output/context triplets and obtaining metrics relating to a number of different aspects of our RAG pipeline.

We'll be evaluating on every core metric today, but in order to do that - we'll need to create a test set. Luckily for us, Ragas can do that directly!

### Synthetic Test Set Generation

We can leverage Ragas' [`Synthetic Test Data generation`](https://docs.ragas.io/en/stable/concepts/testset_generation.html) functionality to generate our own synthetic QC pairs - as well as a synthetic ground truth - quite easily!

In [23]:
loader = PyMuPDFLoader(
    "https://d1lamhf6l6yk6d.cloudfront.net/uploads/2021/08/The-pmarca-Blog-Archives.pdf",
)

eval_documents = loader.load()

text_splitter_eval = RecursiveCharacterTextSplitter(
    chunk_size = 600,
    chunk_overlap = 50
)

eval_documents = text_splitter_eval.split_documents(eval_documents)

####❓ Question #2:

Why is it important to split our documents using different parameters when creating our synthetic data?

--> It is important to split our documents using different parameters when creating our synthetic data because it allows us to create a more diverse and representative test set. If we used the same parameters for splitting our documents, we would likely end up with a test set that is too similar to our training data, and our metrics would not be able to accurately reflect the performance of our RAG pipeline. By using different parameters for splitting our documents, we can create a more diverse set of questions and answers, which can help us to better understand the strengths and weaknesses of our RAG pipeline.

In [24]:
len(eval_documents)

624

> NOTE: 🛑 Running this cell as presented will incur a charge of ~$3USD from OpenAI usage. Most of this cost is produced by the Synthetic Data Generation step. **YOU CAN SKIP THIS STEP BY LOADING THE `.csv` DIRECTLY FROM OUR REPOSITORY.** 🛑

#### Optional: SDG for Evaluation

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

generator_llm = ChatOpenAI(model="gpt-3.5-turbo")
critic_llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings()

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.5,
    multi_context: 0.4,
    reasoning: 0.1
}

num_qa_pairs = 20 # You can reduce the number of QA pairs to 5 if you're experiencing rate-limiting issues

testset = generator.generate_with_langchain_docs(eval_documents, num_qa_pairs, distributions)
testset.to_pandas()

Let's look at the output and see what we can learn about it!

In [ ]:
testset.test_data[0]

In [25]:
testset_df = testset.to_pandas()
testset_df.to_csv("testset.csv")

#### PREFERRED: Download `.csv` from DataRepository

In [26]:
!git clone https://github.com/AI-Maker-Space/DataRepository.git

Cloning into 'DataRepository'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 87 (delta 24), reused 28 (delta 8), pack-reused 8 (from 1)
Receiving objects: 100% (87/87), 70.09 MiB | 26.03 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [27]:
!mv DataRepository/testset.csv .

### Generating Responses with RAG Pipeline

Now that we have some QC pairs, and some ground truths, let's evaluate our RAG pipeline using Ragas.

The process is, again, quite straightforward - thanks to Ragas and LangChain!

Let's start by extracting our questions and ground truths from our create testset.

We can start by converting our test dataset into a Pandas DataFrame.

In [28]:
import pandas as pd

test_df = pd.read_csv("testset.csv")

In [29]:
test_df

,Unnamed: 0,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,0,How does the tendency to avoid inconsistency c...,['Five: Inconsistency-Avoidance Tendency\n[Peo...,The tendency to avoid inconsistency contribute...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
1,1,What are some of the challenges faced by start...,['structure that any established company has.\...,"In a startup, it is easy for the code not to g...",simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
2,2,What factors should be considered when decidin...,['Part 2: Skills and education\n[Please read m...,The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
3,3,What should be valued when evaluating candidat...,"[""How to hire the best people you've\never wor...",The answer to given question is not present in...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
4,4,What are the consequences of not raising enoug...,['Here’s why you shouldn’t do that:\nWhat are ...,Not raising enough money risks the survival of...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
5,5,How does Structured Procrastination suggest us...,['like?\nStructured procrastination\nThis is a...,Structured Procrastination suggests that inste...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
6,6,What analogy is used to describe the layers of...,['as if it’s an onion. Just like you peel an o...,The analogy used to describe the layers of ris...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
7,7,How can Structured Procrastination be used to ...,['like?\nStructured procrastination\nThis is a...,Structured Procrastination suggests that inste...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
8,8,How is the quality of a startup's product defi...,['Let’s start by deXning terms.\nThe caliber o...,The quality of a startup's product in the tech...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True
9,9,What role can a campus computer lab play in he...,"['undergrads to do some of the work, and being...",A campus computer lab can play a role in helpi...,simple,[{'source': 'https://d1lamhf6l6yk6d.cloudfront...,True


In [30]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

Now we'll generate responses using our RAG pipeline using the questions we've generated - we'll also need to collect our retrieved contexts for each question.

We'll do this in a simple loop to see exactly what's happening!

In [31]:
answers = []
contexts = []

for question in test_questions:
  response = retrieval_augmented_qa_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])

Now we can wrap our information in a Hugging Face dataset for use in the Ragas library.

In [32]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

Let's take a peek and see what that looks like!

In [33]:
response_dataset[0]

{'question': 'How does the tendency to avoid inconsistency contribute to people being reluctant to change?',
 'answer': "The tendency to avoid inconsistency contributes to people being reluctant to change because it manifests as a reluctance to alter established habits and beliefs. This avoidance is linked to the brain's anti-change tendency, which maintains previous conclusions, loyalties, reputational identity, and commitments. As a result, individuals may resist new ideas or changes, preferring to stay within their existing frameworks rather than face the discomfort of inconsistency.",
 'contexts': ['Five: Inconsistency-Avoidance Tendency\n[People are] reluctant to change, which is a form of inconsistency\navoidance. We see this in all human habits, constructive and',
  'less brain-blocked by its previous conclusions…\nOne corollary of Inconsistency-Avoidance Tendency is that a per-\nson making big sacriXces in the course of assuming a new identity',
  '[T]ending to be maintained in

# 🤝 Breakout Room Part #2

## Task 1: Evaluating our Pipeline with Ragas

Now that we have our response dataset - we can finally get into the "meat" of Ragas - evaluation!

First, we'll import the desired metrics, then we can use them to evaluate our created dataset!

Check out the specific metrics we'll be using in the Ragas documentation:

- [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
- [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
- [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
- [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
- [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)

See the accompanied presentation for more in-depth explanations about each of the metrics!

In [34]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

All that's left to do is call "evaluate" and away we go!

In [35]:
results = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

In [36]:
results

{'faithfulness': 0.5899, 'answer_relevancy': 0.7218, 'context_recall': 0.6360, 'context_precision': 0.6637, 'answer_correctness': 0.5688}

In [37]:
results_df = results.to_pandas()
results_df

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,How does the tendency to avoid inconsistency c...,The tendency to avoid inconsistency contribute...,[Five: Inconsistency-Avoidance Tendency\n[Peop...,The tendency to avoid inconsistency contribute...,1.000000,0.950362,0.500000,0.805556,0.463127
1,What are some of the challenges faced by start...,Some challenges faced by startups in establish...,[ied and determined. Sales calls get made. The...,"In a startup, it is easy for the code not to g...",1.000000,0.991209,1.000000,1.000000,0.564592
2,What factors should be considered when decidin...,I don't know.,[including your formal education. So I will st...,The answer to given question is not present in...,0.000000,0.000000,1.000000,0.000000,0.195204
3,What should be valued when evaluating candidat...,"When evaluating candidates for a startup, it i...",[priate for your particular startup.\nWith a w...,The answer to given question is not present in...,0.666667,1.000000,1.000000,0.000000,0.179588
4,What are the consequences of not raising enoug...,Not raising enough money risks the survival of...,[Here’s why you shouldn’t do that:\nWhat are t...,Not raising enough money risks the survival of...,0.666667,0.966323,0.333333,0.833333,0.502281
5,How does Structured Procrastination suggest us...,Structured Procrastination suggests that inste...,[standing.)\nThe gist of Structured Procrastin...,Structured Procrastination suggests that inste...,1.000000,0.957872,1.000000,0.916667,0.994430
6,What analogy is used to describe the layers of...,The analogy used to describe the layers of ris...,[as if it’s an onion. Just like you peel an on...,The analogy used to describe the layers of ris...,1.000000,1.000000,1.000000,0.916667,0.891681
7,How can Structured Procrastination be used to ...,Structured Procrastination can be used to one'...,[standing.)\nThe gist of Structured Procrastin...,Structured Procrastination suggests that inste...,1.000000,0.987979,0.500000,0.805556,0.992031
8,How is the quality of a startup's product defi...,The quality of a startup's product in the cont...,[The quality of a startup’s pr\nproduct\noduct...,The quality of a startup's product in the tech...,1.000000,0.993645,0.000000,1.000000,0.651053
9,What role can a campus computer lab play in he...,A campus computer lab can help undergraduates ...,[What should I do while I’m in school?\nI’m a ...,A campus computer lab can play a role in helpi...,0.375000,0.981524,0.333333,0.750000,0.902415


## Task : Testing OpenAI's Claim

Now that we've seen how our retriever can impact the performance of our RAG pipeline - let's see how changing our embedding model impacts performance.

####🏗️ Activity #1:

Please provide markdown, or code comments, to explain which each of the following steps are doing!

In [38]:
# Step 1: Initialize OpenAI Embeddings model
# This step creates an instance of OpenAIEmbeddings using the "text-embedding-3-small" model.
# The text-embedding-3-small model is a newer, more advanced embedding model from OpenAI
# that is expected to provide better performance compared to older models.
te3_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [39]:
# Step 2: Create a new Qdrant collection
# This step creates a new collection in the Qdrant vector database specifically for the 
# text-embedding-3-small model. The collection is named by appending "TE3" to the existing
# COLLECTION_NAME. It sets up the vector configuration with a size of 1536 (matching the 
# output dimension of the new embedding model) and uses cosine distance for similarity calculations.
qdrant_client.create_collection(
    collection_name=COLLECTION_NAME+"TE3",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

# Step 3: Initialize QdrantVectorStore
# This step creates a new QdrantVectorStore instance, connecting it to the newly created
# Qdrant collection and associating it with the text-embedding-3-small embeddings model.
qdrant_vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME+"TE3",
    embedding=te3_embeddings,
)

# Step 4: Add documents to the vector store
# This step adds the previously processed documents to the new vector store, creating
# embeddings using the text-embedding-3-small model.
qdrant_vector_store.add_documents(documents)

['6372b733decf4897b61af855eb0a40a2',
 'c940ff833d7048d3915cd09bf233fc40',
 'a36b038e19144569a3ea40568604ffe1',
 'ed43d54d7d6d449aadc2d0914104d049',
 '7128f3d74e2a407eb7a9475d0ea9cf3d',
 '9ce6e3fd7a18457787480895ddc33914',
 '781c0f514ffc4adda2c23d3bb56f52cb',
 'e13bf396848c403a88a0649fab2df78b',
 '47a5c8310e354f299d1946114e897f21',
 '1a1c2a69386044e3a760e66ce6da03f8',
 '678c75b7984c4a0f91821a75f72b0dcf',
 '7375cd8707184ed9bbc3bcd5b81b8160',
 'c8eeffa80c524f9597c7a1a3d6192fbe',
 '502ba2c71ac9403ebe05c10698e1da3b',
 '6530e625030743d7b7a6524ba8bffe8f',
 'cd3c6621be804b1997ec72bce3c90132',
 'bd3634b5ecca48fb8d26fd9ae2a75f75',
 'c10691eb37dc483b913934c29c671547',
 '2df1e1e637684d9c9c838f59accdedab',
 '37e140588987498fa5bdd3fc8c7fe83b',
 '67fa99b571fa4476abd10530238befdc',
 '864e6eac00e54fdc8e0a80fd32c9888b',
 '011675823d4148898a7131ecfce07b6e',
 '4d64d3a56d5e4221b777821f0ee0855c',
 '56f63241707b431880962b52c1bb12fd',
 'd40f82630d2b4004958a2278340aa929',
 'ca75a1b8f61446b18f31c43b98f8aafa',
 

In [40]:
# Step 5: Create a retriever
# This step creates a retriever object from the Qdrant vector store, which will be used
# to fetch relevant documents based on query embeddings.
te3_retriever = qdrant_vector_store.as_retriever()

In [41]:
from langchain.chains.combine_documents import create_stuff_documents_chain

# Step 6: Create a document chain
# This step creates a chain that combines retrieved documents with the query to generate
# an answer, using the previously defined LLM and prompt.
document_chain = create_stuff_documents_chain(primary_qa_llm, retrieval_qa_prompt)

In [42]:
from langchain.chains import create_retrieval_chain

# Step 7: Create a retrieval chain
# This step combines the retriever and the document chain to create a complete retrieval chain,
# which will handle the entire process from query to answer generation.
te3_retrieval_chain = create_retrieval_chain(te3_retriever, document_chain)

In [43]:
# Step 8: Process test questions
# This step iterates through the test questions, uses the new retrieval chain to generate
# answers, and stores both the answers and the contexts used to generate them.
answers = []
contexts = []

for question in test_questions:
  response = te3_retrieval_chain.invoke({"input" : question})
  answers.append(response["answer"])
  contexts.append([context.page_content for context in response["context"]])

In [44]:
# Step 9: Create a new dataset for advanced retrieval results
# This step creates a new dataset from the answers and contexts generated by the advanced
# retrieval setup. It uses the test_questions, answers, contexts, and test_groundtruths to
# create a new dataset that can be used to evaluate the advanced retrieval setup.   
te3_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [45]:
# Step 10: Evaluate the advanced retrieval results
# This step evaluates the advanced retrieval results using the same metrics as before.
# It uses the evaluate function to calculate the metrics for the new retrieval setup.
te3_advanced_retrieval_results = evaluate(te3_response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

In [46]:
# Prints the results of the advanced retrieval evaluation
te3_advanced_retrieval_results

{'faithfulness': 0.6877, 'answer_relevancy': 0.9721, 'context_recall': 0.6228, 'context_precision': 0.6374, 'answer_correctness': 0.6726}

In [47]:

# Step 11: Create dataframes for comparison
# This step creates two dataframes, df_baseline and df_comparison, to store the results of the
# baseline and comparison evaluations. It uses the items method to extract the metric names
# and their corresponding values from the results, and then merges these dataframes to compare
# the baseline and comparison results.
df_baseline = pd.DataFrame(list(results.items()), columns=['Metric', 'ADA'])
df_comparison = pd.DataFrame(list(te3_advanced_retrieval_results.items()), columns=['Metric', 'TE3'])

df_merged = pd.merge(df_baseline, df_comparison, on='Metric')

df_merged['Baseline -> TE3'] = df_merged['TE3'] - df_merged['ADA']

df_merged

,Metric,ADA,TE3,Baseline -> TE3
0,faithfulness,0.589912,0.687699,0.097787
1,answer_relevancy,0.721753,0.972102,0.250349
2,context_recall,0.635965,0.622807,-0.013158
3,context_precision,0.663743,0.637427,-0.026316
4,answer_correctness,0.568782,0.672626,0.103844


####❓ Question #3:

Do you think, in your opinion, `text-embedding-3-small` is significantly better than `ada`? 

--> No, I don't think that `text-embedding-3-small` is significantly better than `ada`. I think that both embedding models are quite good, and the differences in performance between them are marginal. The text-embedding-3-small model is a newer, more advanced embedding model from OpenAI that is expected to provide better performance compared to older models. But I think that the choice of embedding model should be based on the specific requirements of the application and the trade-offs between performance and cost.

## Task 5: Selecting an Advanced Retriever and Evaluating

#### 🏗️ Activity #2

While the changes that occured due to modifying the embedding model were desirable - you're now tasked with improving `context_recall`, or `context_precision` (or both!).

You'll follow these steps:

1. Reason about this list of Advanced Retrieval methods:
  - [Contextual Compression (Reranker)](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/contextual_compression/)
  - [MultiQueryRetriever](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/MultiQueryRetriever/)
  - [Parent Document Retriever](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/parent_document_retriever/)
2. Select the method you think will be the most performant.
3. Implement that method.
4. Create a LCEL chain that utlizes the new Retriever method.
5. Evaluate this LCEL and compare to the TE3 results.

> NOTE: We will spend more time in Session 14 diving into advanced retrieval methods, this activity is only to serve as a basic introduction to the idea of component-wise improvements and how they might impact metrics.

In [48]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(primary_qa_llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=te3_retriever
)

compressed_chain = create_retrieval_chain(compression_retriever, document_chain)

compressed_answers = []
compressed_contexts = []

for question in test_questions:
  response = compressed_chain.invoke({"input" : question})
  compressed_answers.append(response["answer"])
  compressed_contexts.append([context.page_content for context in response["context"]])

compressed_response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : compressed_answers,
    "contexts" : compressed_contexts,
    "ground_truth" : test_groundtruths
})

compressed_advanced_retrieval_results = evaluate(compressed_response_dataset_advanced_retrieval, metrics)
compressed_advanced_retrieval_results

Evaluating:   0%|          | 0/95 [00:00<?, ?it/s]

{'faithfulness': 0.6714, 'answer_relevancy': 0.9632, 'context_recall': 0.4912, 'context_precision': 0.6784, 'answer_correctness': 0.5523}

In [49]:
df_te3 = pd.DataFrame(list(te3_advanced_retrieval_results.items()), columns=['Metric', 'TE3'])
df_compressed = pd.DataFrame(list(compressed_advanced_retrieval_results.items()), columns=['Metric', 'Compressed'])

df_merged = pd.merge(df_te3, df_compressed, on='Metric')

df_merged['TE3 -> Compressed'] = df_merged['Compressed'] - df_merged['TE3']

df_merged

,Metric,TE3,Compressed,TE3 -> Compressed
0,faithfulness,0.687699,0.671436,-0.016263
1,answer_relevancy,0.972102,0.963186,-0.008916
2,context_recall,0.622807,0.491228,-0.131579
3,context_precision,0.637427,0.678363,0.040936
4,answer_correctness,0.672626,0.552311,-0.120314


--> I chose the Contextual Compression Retriever because it uses an LLM to compress the retrieved contexts, which can help to improve the recall of the retrieved contexts. However, it may not be as effective for improving the precision of the retrieved contexts. Based on the results, it seems that the Contextual Compression Retriever is not as effective as the TE3 retriever at recalling relevant context. The Contextual Compression Retriever is more effective at improving the precision of the retrieved contexts, but it is not as effective at recalling relevant context.

#### 🚧 BONUS CHALLENGE 🚧

> NOTE: Completing this challenge will provide full marks on the assignment, regardless of the complete of the notebook. You do not need to complete this in the notebook for full marks.

##### **MINIMUM REQUIREMENTS**:

1. Baseline `LCEL RAG` Application using `NAIVE RETRIEVAL`
2. Baseline Evaluation using `RAGAS METRICS`
  - [Faithfulness](https://docs.ragas.io/en/stable/concepts/metrics/faithfulness.html)
  - [Answer Relevancy](https://docs.ragas.io/en/stable/concepts/metrics/answer_relevance.html)
  - [Context Precision](https://docs.ragas.io/en/stable/concepts/metrics/context_precision.html)
  - [Context Recall](https://docs.ragas.io/en/stable/concepts/metrics/context_recall.html)
  - [Answer Correctness](https://docs.ragas.io/en/stable/concepts/metrics/answer_correctness.html)
3. Implement a `SEMANTIC CHUNKING STRATEGY`.
4. Create an `LCEL RAG` Application using `SEMANTIC CHUNKING` with `NAIVE RETRIEVAL`.
5. Compare and contrast results.

##### **SEMANTIC CHUNKING REQUIREMENTS**:

Chunk semantically similar (based on designed threshold) sentences, and then paragraphs, greedily, up to a maximum chunk size. Minimum chunk size is a single sentence.

Have fun!